In [ ]:
import numpy as np 
from tensorflow import keras
import os
import cv2
import keras
import tensorflow as tf
from keras import applications
from keras.preprocessing import image as image_utils
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D,MaxPooling2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Flatten
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
from sklearn.metrics import roc_auc_score
import tensorflow.keras.backend as K

def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)


def load_and_scale(img):
  image = image_utils.load_img(img, target_size = ())

train_labels = []
def load_images_from_folder(folder):
    images = []
    
    for idx, filename in enumerate(os.listdir(folder)):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)          

            temp = filename[5:]
            temp = int(temp[:2]) 
                      
            train_labels.append(temp)

    return images

def make_pairs(images,labels):
  pairImages = []
  pairLabels = []
  numClasses = len(np.unique(labels))
  idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
  for idxA in range(len(images)):
    currentImage = images[idxA]
    label = int(labels[idxA])
    
    idxB = np.random.choice(idx[int(label)])
    posImage = images[idxB]
    posImage = images[idxB]

    pairImages.append([currentImage, posImage])
    pairLabels.append([1])

    negIdx = np.where(labels != label)[0]
    negImage = images[np.random.choice(negIdx)]

    pairImages.append([currentImage, negImage])
    pairLabels.append([0])

  return (np.array(pairImages), np.array(pairLabels))

In [ ]:

train_dataset = []
test_dataset = []
train_dataset = load_images_from_folder("/content/drive/MyDrive/train3")



In [ ]:
for i in range(len(train_dataset)):
  train_dataset[i] = cv2.resize(train_dataset[i], dsize=(100, 100), interpolation=cv2.INTER_CUBIC)  
  train_dataset[i] = keras.applications.xception.preprocess_input(
    train_dataset[i], data_format=None
)

train_dataset_np = np.asarray(train_dataset).astype(np.float32)


train_labels_np = np.asarray(train_labels).astype(np.int)


label_dict = {99:5, 98:13, 97:25, 96:32,94:34,93:36,92:38,91:40,90:48,89:50,88:52,86:57,85:60,84:61,83:65,80:76,79:78}
train_labels_np_2 = np.copy(train_labels_np)
for k, v in label_dict.items(): train_labels_np_2[train_labels_np==k] = v





In [ ]:
import itertools
b = list(itertools.combinations(train_labels_np_2,2))
pairLabels = []
for i in range(len(b)):
  if b[i][0]==b[i][1]:
    pairLabels.append(1)
  else:
    pairLabels.append(0)


pairLabels = np.array(pairLabels)
pairLabels = pairLabels[:30000]

pairImages = list(itertools.combinations(train_dataset_np,2))
pairImages = pairImages[:30000]
pairImages_np = np.array(pairImages)


In [ ]:
def make_pairs2(images,labels):
  pairImages = []
  pairLabels = []
  temp=[]
  temp = list(itertools.combinations(labels,2))
  for i in range(len(temp)):
    if temp[i][0] == temp[i][1]:
      pairLabels.append(1)
    else:
      pairLabels.append(0)
  
  pairImages = list(itertools.combinations(images,2))
  return (np.array(pairImages), np.array(pairLabels))

In [ ]:
inp_shape=(100,100,3)
def build_siamese_model2(inputShape = inp_shape, embeddingDim=128):
  # specify the inputs for the feature extractor network
  inputs = Input(inputShape)
  # define the first set of CONV => RELU => POOL => DROPOUT layers
  x = Conv2D(64, (2, 2), padding="same", activation="relu")(inputs)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Dropout(0.3)(x)
  # second set of CONV => RELU => POOL => DROPOUT layers
  x = Conv2D(128, (2, 2), padding="same", activation="relu")(x)
  x = MaxPooling2D(pool_size=2)(x)
  x = Dropout(0.3)(x)
  x = Conv2D(256, (2, 2), padding="same", activation="relu")(x)
  x = MaxPooling2D(pool_size=2)(x)
  x = Dropout(0.3)(x)
 
 	# prepare the final outputs
  pooledOutput = GlobalAveragePooling2D()(x)
  outputs = Dense(embeddingDim)(pooledOutput)
  # build the model
  model = Model(inputs, outputs)
	# return the model to the calling function
  return model


def euclidean_distance(vectors):
  (featsA, featsB) = vectors

  sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
  return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def plot_training(H):
	# construct a plot that plots and saves the training history
  plt.style.use("ggplot")
  plt.figure()
  plt.plot(H.history["loss"], label="train_loss")
  plt.plot(H.history["val_loss"], label="val_loss")
  plt.plot(H.history["accuracy"], label="train_acc")
  plt.plot(H.history["val_accuracy"], label="val_acc")
  plt.plot(H.history["val_accuracy"], label="val_acc")
  plt.title("Training Loss and Accuracy")
  plt.xlabel("Epoch #")
  plt.ylabel("Loss/Accuracy")
  plt.legend(loc="lower left")
  

In [ ]:
#model3 = keras.models.load_model('/content/drive/MyDrive/model2.h5')
imgA = Input(inp_shape)
imgB = Input(inp_shape)
feature = build_siamese_model2()

featsA = feature(imgA)
featsB = feature(imgB)



In [ ]:
from keras.layers import Lambda
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)
model3 = Model(inputs=[imgA, imgB], outputs=outputs)

In [ ]:
model3.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 128)          197952      input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           model[0][0]                

In [ ]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model3.fit([pairImages_np[:,0], pairImages_np[:,1]], pairLabels[:], validation_split=0.2, epochs=40)

Epoch 1/4
250/250 [==============================] - 48s 59ms/step - loss: 0.0815 - accuracy: 0.9879 - val_loss: 0.0939 - val_accuracy: 0.9915
Epoch 2/4
250/250 [==============================] - 14s 57ms/step - loss: 0.0353 - accuracy: 0.9894 - val_loss: 0.0330 - val_accuracy: 0.9915
Epoch 3/4
250/250 [==============================] - 15s 58ms/step - loss: 0.0228 - accuracy: 0.9899 - val_loss: 0.0212 - val_accuracy: 0.9915
Epoch 4/4
250/250 [==============================] - 15s 58ms/step - loss: 0.0184 - accuracy: 0.9906 - val_loss: 0.0136 - val_accuracy: 0.9915


In [ ]:
model3.save("/content/drive/MyDrive/model4.h5")

In [ ]:
import glob


test_dataset2 = []
filenames = [img for img in glob.glob('/content/drive/MyDrive/test/test/*.png')]

filenames.sort()

for img in filenames:
  n=cv2.imread(img)
  test_dataset2.append(n)

for i in range(len(test_dataset2)):
  test_dataset2[i] = cv2.resize(test_dataset2[i], dsize=(100, 100), interpolation=cv2.INTER_CUBIC)  
  test_dataset2[i] = keras.applications.xception.preprocess_input(
    test_dataset2[i], data_format=None
)
  #test_dataset2[i] =  test_dataset2[i]/255 

test_dataset_np = np.asarray(test_dataset2).astype(np.float32)

  

In [ ]:
test_dataset_np2 = []
for i in range(len(test_dataset_np)):
  test_dataset_np2.append(np.expand_dims(test_dataset_np[i], axis=0))

test_dataset_np2 = np.array(test_dataset_np2)

In [ ]:
test_dataset_np2.shape

(204, 1, 100, 100, 3)

In [ ]:
import pandas as pd


In [ ]:
predictions = []
for i in range(1):
  print(i)
  for j in range(len(test_dataset_np)):    

    preds = model3.predict([test_dataset_np2[i][0], test_dataset_np2[j]])
    predictions.append(preds)



In [ ]:
df = pd.DataFrame(columns=['score'])
df['score'] = predictions
df.to_csv('/content/drive/MyDrive/test_np.csv')
